In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib as mpl


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.copy_on_write = True

In [2]:
con_r = sqlite3.Connection('./radio_ads.db')
cur_r = con_r.cursor()
con_tv = sqlite3.Connection('../tv_sql_ver/tv_ads.db')
cur_tv = con_r.cursor()

In [3]:
# Zmienne do wyciągnięcia danych z DB
rok_s = 2021
rok_e = 2022
mc_s = 1
mc_e = 12
dzi_s = 1
dzi_e = 31

# Zmienne do określenia porównań r/r
rok_s_rdr = 2021
rok_e_rdr = 2022
mc_s_rdr = 'Kwiecień'
mc_e_rdr = 'Kwiecień'
dzi_s_rdr = 1
dzi_e_rdr = 31

# Zmienne do okreśłenia porównań m/m
rok_s_mdm = rok_e_mdm = 2022
mc_s_mdm = 'Marzec'
mc_e_mdm = 'Kwiecień'
dzi_s_mdm = 1
dzi_e_mdm = 30

# Zmienne do wyciągnięćia porównań t/t
rok_s_tdt = rok_e_tdt = 2022
mc_s_tdt = 'Kwiecień'
mc_e_tdt = 'Kwiecień'
dzi_s_tdt = 25
dzi_e_tdt = 30

In [4]:
query_r = f'''SELECT "data_czas"."data" AS "data", "dzien", "dzien_tyg_nr" AS 'd_tyg_nr', "dzien_tyg", 
"tydzien", "miesiac_nr" AS "m_nr", "miesiac", "rok", "kod_rek" AS "kod_reklamy", "brand", 
"submedium", "nadawca", "zasieg", "godz_blok_rek" AS "godz_bloku", "daypart", 
"dl_ujednolicona" AS "dl_spotu", "typ_produktu", "koszt", "typ_rek", "l_emisji" AS "ilosc"
FROM "spoty"
JOIN "data_czas" ON "data_czas"."data" = "spoty"."data"
JOIN "brandy" ON "brandy"."id" = "spoty"."brand_id"
JOIN "submedia" ON "submedia"."id" = "spoty"."submedium_id"
JOIN "nadawcy" ON "nadawcy"."id" = "submedia"."nadawca_id"
JOIN "zasiegi" ON "zasiegi"."id" = "submedia"."zasieg_id"
JOIN "dayparty" ON "dayparty"."id" = "spoty"."daypart_id"
JOIN "dl_ujednolicone" ON "dl_ujednolicone"."id" = "spoty"."dl_ujednolicona_id"
JOIN "typy_produktu" ON "typy_produktu"."id" = "spoty"."typ_produktu_id"
JOIN "dni_tyg" ON "dni_tyg"."id" = "data_czas"."dzien_tyg_nr"
JOIN "miesiace" ON "miesiace"."id" = "data_czas"."miesiac_nr"
JOIN "bloki_rek" ON "bloki_rek"."id" = "spoty"."blok_rek_id"
JOIN "kody_rek" ON "kody_rek"."id" = "spoty"."kod_rek_id"
JOIN "typy_rek" ON "typy_rek"."id" = "spoty"."typ_rek_id"
WHERE "miesiac_nr" BETWEEN {mc_s} AND {mc_e} AND "rok" BETWEEN {rok_s} AND {rok_e} 
AND "dzien" BETWEEN {dzi_s} AND {dzi_e};'''


query_tv = f'''SELECT data_czas.data AS data, dzien, dzien_tyg_nr AS d_tyg_nr, dzien_tyg,
tydzien, miesiac_nr AS m_nr, miesiac, rok, kod_rek AS kod_reklamy, brand, kanal, 
kanal_gr, czas, dlugosc, daypart, grp AS ilosc
FROM spoty
JOIN miesiace ON miesiace.id = data_czas.miesiac_nr
JOIN dni_tyg ON dni_tyg.id = data_czas.dzien_tyg_nr
JOIN data_czas ON data_czas.data = spoty.data
JOIN kody_rek ON kody_rek.id = spoty.kod_rek_id
JOIN dlugosci ON dlugosci.id = spoty.dlugosc_id
JOIN dayparty ON dayparty.id = spoty.daypart_id
JOIN brandy ON brandy.id = spoty.brand_id
JOIN kanaly ON kanaly.id = spoty.kanal_id
JOIN kanaly_gr ON kanaly_gr.id = kanaly.kanal_gr_id
WHERE m_nr BETWEEN {mc_s} AND {mc_e} AND rok BETWEEN {rok_s} AND {rok_e} 
AND dzien BETWEEN {dzi_s} AND {dzi_e};'''

In [5]:
df_r = pd.read_sql(sql=query_r, con=con_r)
df_r.head(1)

,data,dzien,d_tyg_nr,dzien_tyg,tydzien,m_nr,miesiac,rok,kod_reklamy,brand,submedium,nadawca,zasieg,godz_bloku,daypart,dl_spotu,typ_produktu,koszt,typ_rek,ilosc
0,2021-01-01,1,5,Piątek,0,1,Styczeń,2021,22082881,EURO RTV AGD,PR 1,POLSKIE RADIO,krajowe,7:00-7:29,do 9,30,GRUPA,12300,reklama,1


In [6]:
df_tv = pd.read_sql(sql=query_tv, con=con_tv)
df_tv.head(1)

,data,dzien,d_tyg_nr,dzien_tyg,tydzien,m_nr,miesiac,rok,kod_reklamy,brand,kanal,kanal_gr,czas,dlugosc,daypart,ilosc
0,2022-01-01,1,6,Sobota,0,1,Styczeń,2022,290732,MEDIA EXPERT,TVP1,TVP,02:00:04,15,off,0.95


In [7]:
# dziennie_r = df_r[['brand', 'submedium', 'ilosc', 'dzien', 'dzien_tyg', 'data', 'rok', 'miesiac']]
# dziennie_r.loc[:, 'dzien_tyg'] = dziennie_r['dzien_tyg'].str[:2]
# dziennie_r['medium'] = 'radio'

# dziennie_tv = df_tv[['brand', 'ilosc', 'dzien', 'dzien_tyg', 'data', 'rok', 'miesiac']]
# dziennie_tv.loc[:, 'dzien_tyg'] = dziennie_tv['dzien_tyg'].str[:2]
# dziennie_tv['medium'] = 'tv'

In [8]:
# stacje = ['ESKA Wrocław', 'ZET', 'RMF FM', 'TOK FM', 'PR 1']
# rok = (2023, 2022)
# miesiac = 'Styczeń'
# dzien = 15
# filtr = (dziennie_r['rok'].isin(rok)) & (dziennie_r['miesiac'] == miesiac) & (dziennie_r['submedium'].isin(stacje)) & (dziennie_r['dzien'] <= dzien) 
# dziennie_r = dziennie_r.loc[filtr]
# dziennie_r.set_index(['submedium', 'brand'], inplace=True)
# dziennie_r = dziennie_r.sort_index(ascending=True)
# tp_r = dziennie_r.pivot_table(values='ilosc', index=['submedium', 'brand'], columns=['rok', 'miesiac','dzien', 'dzien_tyg'], aggfunc='sum', fill_value=0)

# filtr_t = (dziennie_tv['rok'].isin(rok)) & (dziennie_tv['miesiac'] == miesiac) & (dziennie_tv['dzien'] <= dzien)
# dziennie_tv = dziennie_tv[filtr_t]
# dziennie_tv.set_index(['brand'], inplace=True)
# dziennie_tv = dziennie_tv.sort_index(ascending=True)
# tp_tv = dziennie_tv.pivot_table(values='ilosc', index=['brand'], columns=['rok', 'miesiac', 'dzien', 'dzien_tyg'], aggfunc='sum', fill_value=0)

In [9]:
# tp_r

In [10]:
# tp_tv.style.format('{:.0f}')

In [11]:
# tp_r.diff(axis=1, periods=15).style.format('{:.0f}', na_rep='-')

In [12]:
# tp_tv.diff(axis=1, periods=15).style.format('{:.0f}', na_rep='-')

In [13]:
# def highlight_max(s):
#     '''
#     highlight the maximum in a Series green.
#     '''
#     is_max = s == s.max()
#     return ['background-color: green' if v else '' for v in is_max]

# def highlight_min(s):
#     '''
#     highlight the maximum in a Series red.
#     '''
#     is_min = s == s.min()
#     return ['background-color: brown' if v else '' for v in is_min]

# tp_r.pct_change(axis='columns', periods=15).style.format('{:.0%}', na_rep='-').apply(highlight_min)

In [14]:
# tp_tv.pct_change(axis=1, periods=15).style.format('{:.0%}', na_rep='-').apply(highlight_min)

In [17]:
dziennie_r = df_r[['brand', 'submedium', 'ilosc', 'dzien', 'dzien_tyg', 'data', 'rok', 'miesiac']]
dziennie_r.loc[:, 'dzien_tyg'] = dziennie_r['dzien_tyg'].str[:2]
dziennie_r['medium'] = 'radio'

dziennie_t = df_tv[['brand', 'ilosc', 'dzien', 'dzien_tyg', 'data', 'rok', 'miesiac']]
dziennie_t.loc[:, 'dzien_tyg'] = dziennie_t['dzien_tyg'].str[:2]
dziennie_t['medium'] = 'tv'

# r/r
stacje = ['ESKA Wrocław', 'ZET', 'RMF FM', 'TOK FM', 'PR 1']
rok_rdr = (rok_s_rdr, rok_e_rdr)
miesiac_rdr = (mc_s_rdr, mc_e_rdr)
dzien_rdr = dzi_e_rdr

filtr_r_rdr = (dziennie_r['submedium'].isin(stacje)) & (dziennie_r['rok'].isin(rok_rdr)) & (dziennie_r['miesiac'].isin(miesiac_rdr)) \
              & (dziennie_r['dzien'] <= dzien_rdr)
dziennie_r_rdr = dziennie_r.loc[filtr_r_rdr]
dziennie_r_rdr.set_index(['brand', 'medium'], inplace=True)
dziennie_r_rdr = dziennie_r_rdr.sort_index(ascending=True)

filtr_t_rdr = (dziennie_t['rok'].isin(rok_rdr)) & (dziennie_t['miesiac'].isin(miesiac_rdr)) & (dziennie_t['dzien'] <= dzien_rdr)
dziennie_t_rdr = dziennie_t[filtr_t_rdr]
dziennie_t_rdr.set_index(['brand', 'medium'], inplace=True)
dziennie_t_rdr = dziennie_t_rdr.sort_index(ascending=True)

tp_r_rdr = dziennie_r_rdr.pivot_table(values='ilosc', index=['brand', 'medium'], columns=['rok', 'miesiac'], aggfunc='sum')
tp_t_rdr = dziennie_t_rdr.pivot_table(values='ilosc', index=['brand', 'medium'], columns=['rok', 'miesiac'], aggfunc='sum')


# m/m
rok_mdm = (rok_s_mdm, rok_e_mdm)
miesiac_mdm = (mc_s_mdm, mc_e_mdm)
dzien_mdm = (dzi_s_mdm, dzi_e_mdm)

filtr_r_mdm = (dziennie_r['submedium'].isin(stacje)) & (dziennie_r['rok'].isin(rok_mdm)) & (dziennie_r['miesiac'].isin(miesiac_mdm)) \
          & (dziennie_r['dzien'] <= dzien_mdm[1]) & (dziennie_r['dzien'] >= dzien_mdm[0])
dziennie_r_mdm = dziennie_r.loc[filtr_r_mdm]
dziennie_r_mdm.set_index(['brand', 'medium'], inplace=True)
dziennie_r_mdm = dziennie_r_mdm.sort_index(ascending=True)

filtr_t_mdm = (dziennie_t['rok'].isin(rok_mdm)) & (dziennie_t['miesiac'].isin(miesiac_mdm)) & (dziennie_t['dzien'] <= dzien_mdm[1]) \
          & (dziennie_t['dzien'] >= dzien_mdm[0])
dziennie_t_mdm = dziennie_t[filtr_t_mdm]
dziennie_t_mdm.set_index(['brand', 'medium'], inplace=True)
dziennie_t_mdm = dziennie_t_mdm.sort_index(ascending=True)

tp_r_mdm = dziennie_r_mdm.pivot_table(values='ilosc', index=['brand', 'medium'], columns=['rok', 'miesiac'], aggfunc='sum')
tp_t_mdm = dziennie_t_mdm.pivot_table(values='ilosc', index=['brand', 'medium'], columns=['rok', 'miesiac'], aggfunc='sum')


# t/t
rok_tdt = (rok_s_tdt, rok_e_tdt)
miesiac_tdt = (mc_s_tdt, mc_e_tdt)
dzien_tdt = (dzi_s_tdt, dzi_e_tdt)

filtr_r_tdt = (dziennie_r['submedium'].isin(stacje)) & (dziennie_r['rok'].isin(rok_tdt)) & (dziennie_r['miesiac'].isin(miesiac_tdt)) \
          & (dziennie_r['dzien'] <= dzien_tdt[1]) & (dziennie_r['dzien'] >= dzien_tdt[0])
dziennie_r_tdt = dziennie_r.loc[filtr_r_tdt]
dziennie_r_tdt.set_index(['brand', 'medium'], inplace=True)
dziennie_r_tdt = dziennie_r_tdt.sort_index(ascending=True)

filtr_t_tdt = (dziennie_t['rok'].isin(rok_tdt)) & (dziennie_t['miesiac'].isin(miesiac_tdt)) & (dziennie_t['dzien'] <= dzien_tdt[1]) \
          & (dziennie_t['dzien'] >= dzien_tdt[0])
dziennie_t_tdt = dziennie_t[filtr_t_tdt]
dziennie_t_tdt.set_index(['brand', 'medium'], inplace=True)
dziennie_t_tdt = dziennie_t_tdt.sort_index(ascending=True)

tp_r_tdt = dziennie_r_tdt.pivot_table(values='ilosc', index=['brand', 'medium'], columns=['rok', 'miesiac'], aggfunc='sum')
tp_t_tdt = dziennie_t_tdt.pivot_table(values='ilosc', index=['brand', 'medium'], columns=['rok', 'miesiac'], aggfunc='sum')



In [18]:
tp_rdr = tp_r_rdr.merge(tp_t_rdr, how='outer', on=['brand', 'medium'])
tp_rdr = pd.concat([tp_r_rdr, tp_t_rdr], axis=0)
tp_rdr = tp_rdr.groupby(['brand', 'medium']).sum()
tp_rdr['roznica'] = tp_rdr[rok_rdr[1]] - tp_rdr[rok_rdr[0]]
tp_rdr['dynamika'] = tp_rdr[rok_rdr[1]] / tp_rdr[rok_rdr[0]] - 1
tp_rdr.style \
    .format('{:,.0f}', thousands=' ', decimal=',') \
    .format('{:,.1%}', thousands=' ', decimal=",", subset='dynamika')

In [32]:
tp_mdm = tp_r_mdm.merge(tp_t_mdm, how='outer', on=['brand', 'medium'])
tp_mdm = pd.concat([tp_r_mdm, tp_t_mdm], axis=0)
tp_mdm = tp_mdm.groupby(['brand', 'medium']).sum()
tp_mdm['roznica'] = tp_mdm[rok_mdm[1]][miesiac_mdm[1]] - tp_mdm[rok_mdm[0]][miesiac_mdm[0]]
tp_mdm['dynamika'] = tp_mdm[rok_mdm[1]][miesiac_mdm[1]] / tp_mdm[rok_mdm[0]][miesiac_mdm[0]] - 1
tp_mdm.style \
    .format('{:,.0f}', thousands=' ', decimal=',') \
    .format('{:,.1%}', thousands=' ', decimal=",", subset='dynamika')

In [33]:
tp_tdt = tp_r_tdt.merge(tp_t_tdt, how='outer', on=['brand', 'medium'])
tp_tdt = pd.concat([tp_r_tdt, tp_t_tdt], axis=0)
tp_tdt = tp_tdt.groupby(['brand', 'medium']).sum()
tp_tdt['roznica'] = tp_tdt[rok_tdt[1]][miesiac_tdt[1]] - tp_tdt[rok_tdt[0]][miesiac_tdt[0]]
tp_tdt['dynamika'] = tp_tdt[rok_tdt[1]][miesiac_tdt[1]] / tp_mdm[rok_tdt[0]][miesiac_tdt[0]] - 1
tp_tdt.style \
    .format('{:,.0f}', thousands=' ', decimal=',') \
    .format('{:,.1%}', thousands=' ', decimal=",", subset='dynamika')